### 🔑 Setup: API Key Loading and Package Installation

This cell installs necessary libraries and securely loads the Gemini API Key from the environment/ Secrets. This is the foundational step for all subsequent agent interactions.

In [1]:
# Cell 1 — Install + Load Gemini API key from Kaggle Secrets

!pip install -q google-genai python-dotenv gradio

import os
from google.colab import userdata

# load Gemini API key from Kaggle Secrets

api_key = userdata.get("GEMINI_API_KEY")

if not api_key:
    raise ValueError("GEMINI_API_KEY is not defined in Kaggle Secrets.")

os.environ["GEMINI_API_KEY"] = api_key

print("✅ Gemini API Key loaded from Kaggle Secrets.")

✅ Gemini API Key loaded from Kaggle Secrets.


### ⚙️ Core Configuration and LLM Utility

Sets up the Gemini client, defines the model (`gemini-2.5-flash`), and includes the central `llm_call` function. This utility abstracts the LLM interaction, allowing the subsequent agents to focus purely on their reasoning and data flow.### 🧠 MemoryBank: Long-Term State Management

This class implements the **Long Term Memory** concept. It uses a JSON-backed structure to persist the creator's profile, a history of **Past Content Plans** (including detailed **Check-in History**), and identified **Failure Patterns**. This memory is critical for the `MemoryAgent` and `AccountabilityAgent` to learn and provide personalized advice.

In [2]:
# Cell 2: Imports & configuration

from typing import Any, Dict, List
from datetime import datetime, timedelta
import json
import os
import uuid
import re # Added regex import for StrategistAgent

from google import genai
from dotenv import load_dotenv

import gradio as gr

# Load .env if present (harmless)
load_dotenv()

GEMINI_MODEL = "gemini-2.5-flash"


def get_client() -> genai.Client:
    """
    Returns a configured Gemini client.
    Reads API key from GEMINI_API_KEY.
    """
    api_key = os.getenv("GEMINI_API_KEY")
    if not api_key:
        raise RuntimeError("GEMINI_API_KEY is not set. Set it in Cell 1 or via environment.")
    client = genai.Client(api_key=api_key)
    return client


# Global Gemini client (lazy init)
_CLIENT: genai.Client | None = None


def llm_call(prompt: str) -> str:
    """
    Simple helper to call Gemini with a text prompt.
    """
    global _CLIENT
    if _CLIENT is None:
        _CLIENT = get_client()

    resp = _CLIENT.models.generate_content(
        model=GEMINI_MODEL,
        contents=prompt,
    )
    return resp.text


def now_utc() -> datetime:
    return datetime.utcnow()

### 🧠 MemoryBank: Long-Term State Management

This class implements the **Long Term Memory** concept. It uses a JSON-backed structure to persist the creator's profile, a history of **Past Content Plans** (including detailed **Check-in History**), and identified **Failure Patterns**. This memory is critical for the `MemoryAgent` and `AccountabilityAgent` to learn and provide personalized advice.

In [3]:
# Cell 3: MemoryBank – JSON-backed long-term memory (with check-in history support)

class MemoryBank:
    """
    Tiny JSON-backed memory bank.

    Structure:
    {
      "user_profile": {...} or null,
      "past_content": [ContentPlan, ...],
      "failure_patterns": [FailurePattern, ...],
      "accountability_states": [ ... ]
    }

    Each ContentPlan may contain a "checkins": [ {timestamp, completed_ids, blockers, energy, coaching_excerpt}, ... ]
    """

    def __init__(self, path: str = "memory.json"):
        self.path = path
        self.state = {
            "user_profile": None,
            "past_content": [],
            "failure_patterns": [],
            "accountability_states": [],
        }
        self._load()

    def _load(self):
        if os.path.exists(self.path):
            try:
                with open(self.path, "r", encoding="utf-8") as f:
                    self.state = json.load(f)
            except Exception:
                # If corrupted, reset
                self.state = {
                    "user_profile": None,
                    "past_content": [],
                    "failure_patterns": [],
                    "accountability_states": [],
                }

    def _save(self):
        with open(self.path, "w", encoding="utf-8") as f:
            json.dump(self.state, f, indent=2)

    # ---------- User profile ----------

    def get_user_profile(self) -> Dict[str, Any] | None:
        return self.state.get("user_profile")

    def set_user_profile(self, profile: Dict[str, Any]):
        self.state["user_profile"] = profile
        self._save()

    # ---------- Content plans ----------

    def add_content_plan(self, plan: Dict[str, Any]):
        # ensure checkins key exists
        if "checkins" not in plan:
            plan["checkins"] = []
        self.state["past_content"].append(plan)
        self._save()

    def update_content_plan(self, plan_id: str, updates: Dict[str, Any]):
        for idx, plan in enumerate(self.state["past_content"]):
            if plan.get("id") == plan_id:
                # preserve existing checkins if not in updates
                if "checkins" not in updates and "checkins" in plan:
                    updates["checkins"] = plan["checkins"]
                self.state["past_content"][idx].update(updates)
                self._save()
                return self.state["past_content"][idx]
        return None

    def get_content_plan(self, plan_id: str) -> Dict[str, Any] | None:
        for plan in self.state["past_content"]:
            if plan.get("id") == plan_id:
                # guarantee checkins key
                if "checkins" not in plan:
                    plan["checkins"] = []
                    self._save()
                return plan
        return None

    def list_content_plans(self) -> List[Dict[str, Any]]:
        return self.state["past_content"]

    # ---------- Failure patterns ----------

    def list_failure_patterns(self) -> List[Dict[str, Any]]:
        return self.state["failure_patterns"]

    def increment_failure_pattern(self, trigger: str, description: str | None = None):
        patterns = self.state["failure_patterns"]
        for fp in patterns:
            if fp.get("trigger") == trigger:
                fp["evidence_count"] = fp.get("evidence_count", 0) + 1
                break
        else:
            patterns.append(
                {
                    "pattern_id": f"fp_{len(patterns) + 1}",
                    "trigger": trigger,
                    "description": description or trigger,
                    "evidence_count": 1,
                }
            )
        self._save()

    # ---------- Accountability states ----------

    def upsert_accountability_state(self, state: Dict[str, Any]):
        cps_id = state.get("content_plan_id")
        existing = None
        for s in self.state["accountability_states"]:
            if s.get("content_plan_id") == cps_id:
                existing = s
                break
        if existing:
            existing.update(state)
        else:
            self.state["accountability_states"].append(state)
        self._save()

    def get_accountability_state(self, content_plan_id: str) -> Dict[str, Any] | None:
        for s in self.state["accountability_states"]:
            if s.get("content_plan_id") == content_plan_id:
                return s
        return None

    # ---------- Check-in history helper ----------

    def append_checkin(self, plan_id: str, record: Dict[str, Any]):
        plan = self.get_content_plan(plan_id)
        if not plan:
            return False
        if "checkins" not in plan:
            plan["checkins"] = []
        plan["checkins"].append(record)
        self.update_content_plan(plan_id, plan)
        return True

### 🛠️ Custom Tools and Observability (Logging)

Defines two core **Custom Tools** (`search_trends` and `seo_keyword_tool`) simulated using the LLM to research trends and optimize titles/keywords. It also implements the **Logging** framework (`log_event`) for basic observability, allowing users to trace the agent sequence and inputs/outputs.

In [4]:
# Cell 4: Tools – logging, pseudo-search, SEO tool

LOGS: List[Dict[str, Any]] = []


def log_event(agent: str, input_summary: str, output_summary: str):
    LOGS.append(
        {
            "timestamp": now_utc().isoformat(timespec="seconds"),
            "agent": agent,
            "input": input_summary,
            "output": (output_summary or "")[:400],
        }
    )


def get_logs() -> List[Dict[str, Any]]:
    return LOGS


def search_trends(niche: str, platform: str) -> Dict[str, Any]:
    """
    'Search' tool using Gemini as a research assistant.
    In the writeup you can describe this as a tool abstraction.
    """
    prompt = f"""
You are a research assistant for a {platform} content creator in the niche: "{niche}".

1. List 3–5 currently hot themes or topics in this niche.
2. For each, give:
   - A short description
   - The typical hook/angle creators are using
3. Suggest 3 concrete content directions for a solo creator.

Respond clearly in bullet points.
"""
    text = llm_call(prompt)
    log_event("TrendTool", f"niche={niche}, platform={platform}", text)
    return {"raw_summary": text}


def seo_keyword_tool(topic: str, niche: str, platform: str) -> Dict[str, Any]:
    """
    Custom SEO tool: titles, keywords, tags.
    """
    prompt = f"""
You are an SEO assistant for a {platform} creator in the niche "{niche}".

For the topic:
"{topic}"

Generate:
1) 3-5 strongly optimized titles.
2) 1 primary keyword.
3) 5-10 secondary/long-tail keywords.
4) 8-15 tags/hashtags.

Return in this structured markdown form:

TITLES:
- ...

PRIMARY_KEYWORD:
- ...

SECONDARY_KEYWORDS:
- ...

TAGS:
- ...
"""
    text = llm_call(prompt)
    log_event("SEOTool", f"topic={topic}", text)
    return {"raw_seo_text": text}

### 🤖 Multi-Agent System Core

This cell defines the four specialized agents, demonstrating **Multi-Agent System**, **Parallel Agents**, **Sequential Agents**, **Loop Agents**, and the **A2A Protocol**:

1.  **TrendAgent:** Gathers external (simulated) trend data (inputs to Strategist).
2.  **MemoryAgent:** Analyzes historical data from the `MemoryBank` (Parallel to Trend, inputs to Strategist).
3.  **StrategistAgent:** The central brain. It takes the insights from Trend and Memory, uses the SEO **Tool**, and generates a full content plan. It performs an **A2A Handoff** of the plan's tasks to the Accountability Agent.
4.  **AccountabilityAgent:** The **Loop Agent**. It handles user check-ins, updates task status, calculates progress, identifies **Failure Patterns**, and provides **Micro-Coaching** using the LLM.

In [5]:
# Cell 5: Agents – Trend, Memory, Strategist, Accountability (Final Corrected Version)

class BaseAgent:
    def __init__(self, name: str, memory: MemoryBank):
        self.name = name
        self.memory = memory

    def log(self, input_summary: str, output_summary: str):
        log_event(self.name, input_summary, output_summary)


# ---------- Trend Agent ----------

class TrendAgent(BaseAgent):
    """
    Agent A: Researches what's trending in the niche/platform.
    """

    def run(
        self,
        niche: str,
        platform: str
    ) -> Dict[str, Any]:
        result = search_trends(niche, platform)
        self.log(f"niche={niche}, platform={platform}", result["raw_summary"])
        return {
            "trend_summary": result["raw_summary"],
        }


# ---------- Memory Agent ----------

class MemoryAgent(BaseAgent):
    """
    Agent B: Summarizes what has historically worked for this creator.
    """

    def run(self) -> Dict[str, Any]:
        user_profile = self.memory.get_user_profile() or {}
        past_content = self.memory.list_content_plans()
        failure_patterns = self.memory.list_failure_patterns()

        prompt = f"""
You are analyzing historical performance for a solo creator.

USER PROFILE:
{user_profile}

PAST CONTENT PLANS:
{past_content}

FAILURE PATTERNS:
{failure_patterns}

1. Summarize which formats and tones seem to work best.
2. Note any topics/styles that did NOT work well (if any).
3. Summarize common blockers or failure patterns.
4. Give concise recommendations to improve the next content plan.
"""
        text = llm_call(prompt)
        self.log("Analyze memory", text)
        return {
            "memory_insights": text,
        }


# ---------- Strategist Agent ----------

class StrategistAgent(BaseAgent):
    """
    Agent C: Combines trend + memory to create a content plan
    + 48h production tasks.
    """

    def run(
        self,
        trend_result: Dict[str, Any],
        memory_result: Dict[str, Any],
        time_window_hours: int = 48,
    ) -> Dict[str, Any]:
        user_profile = self.memory.get_user_profile() or {}
        platform = user_profile.get("platform", "youtube")
        niche = user_profile.get("niche", "general")

        # Choose topic
        prompt_topic = f"""
You are the Strategist Agent for a solo {platform} creator in niche "{niche}".

TRENDS:
{trend_result['trend_summary']}

MEMORY INSIGHTS:
{memory_result['memory_insights']}

1. Propose ONE best-fit topic for the next content piece.
2. Explain in 3-4 sentences why this topic fits.
3. Suggest a clear angle/hook.

Return:

TOPIC:
...

RATIONALE:
...
"""
        topic_response = llm_call(prompt_topic)
        self.log("Choose topic", topic_response)

        topic_line = "Content idea"
        # Robust extraction: find content between 'TOPIC:' and 'RATIONALE:'
        match = re.search(r"TOPIC:\s*(.*?)\s*RATIONALE:", topic_response, re.DOTALL | re.IGNORECASE)
        if match:
            topic_line = match.group(1).strip()
        else:
            # Fallback to naive extraction if regex fails
            lines = topic_response.splitlines()
            seen_topic = False
            for line in lines:
                if line.strip().upper().startswith("TOPIC:"):
                    seen_topic = True
                    continue
                if seen_topic and line.strip() and not line.strip().upper().startswith("RATIONALE:"):
                    topic_line = line.strip()
                    break

        # SEO tool
        seo_result = seo_keyword_tool(topic_line, niche, platform)

        # Outline & production steps
        prompt_outline = f"""
You are the Strategist Agent.

Create:
1. A 5-10 bullet outline for content with topic "{topic_line}" for {platform}.
2. A 4-6 step list of production tasks (script, record, edit, upload, thumbnail, etc.)
   for a creator who has about {time_window_hours} hours spread over 2 days.

Return clearly with headings:

OUTLINE:
- ...

PRODUCTION_STEPS:
- ...
"""
        outline_text = llm_call(prompt_outline)
        self.log("Outline + steps", outline_text)

        # Simplified fixed task schedule (to avoid JSON parsing issues)
        now = now_utc()
        deadline = now + timedelta(hours=time_window_hours)

        default_tasks = [
            "Draft script",
            "Polish script & finalize outline",
            "Record video/audio",
            "Edit content",
            "Upload & publish with title/tags/thumbnail",
        ]
        tasks: List[Dict[str, Any]] = []
        step_duration = (
            max(time_window_hours // len(default_tasks), 4)
            if time_window_hours >= len(default_tasks)
            else 4
        )

        for idx, desc in enumerate(default_tasks):
            tasks.append(
                {
                    "task_id": f"task_{idx+1}",
                    "description": desc,
                    "status": "pending",
                    # Corrected timespec to seconds for consistency (fixing previous syntax error in unshown versions)
                    "due_time": (now + timedelta(hours=step_duration * (idx + 1))).isoformat(timespec="seconds"),
                    "last_update": now.isoformat(timespec="minutes"),
                    "notes": "",
                }
            )

        plan_id = f"plan_{uuid.uuid4().hex[:8]}"

        content_plan = {
            "id": plan_id,
            "created_at": now.isoformat(timespec="minutes"),
            "topic": topic_line,
            "trend_summary": trend_result["trend_summary"],
            "memory_insights": memory_result["memory_insights"],
            "seo_raw": seo_result["raw_seo_text"],
            "outline_raw": outline_text,
            "tasks": tasks,
            "deadline": deadline.isoformat(timespec="minutes"),
            "status": "active",
            "checkins": [],
        }

        # Store in memory bank
        self.memory.add_content_plan(content_plan)

        # A2A handoff payload for Accountability agent
        handoff_payload = {
            "content_plan_id": plan_id,
            "tasks": tasks,
            "deadline": content_plan["deadline"],
        }

        return handoff_payload


# ---------- Accountability Agent ----------

class AccountabilityAgent(BaseAgent):
    """
    Agent D: Loop-style agent for check-ins, updates tasks,
    learns failure patterns, and gives micro-coaching.

    Here, each Gradio submit simulates one "tick" of the loop.
    """

    def initialize_state(self, handoff: Dict[str, Any]):
        state = {
            "content_plan_id": handoff["content_plan_id"],
            "tasks": handoff["tasks"],
            "deadline": handoff["deadline"],
            "next_check_in": now_utc().isoformat(timespec="minutes"),
            "check_in_interval_hours": 6,
            "completed": False,
        }
        self.memory.upsert_accountability_state(state)

    def run_check_in(
        self,
        content_plan_id: str,
        user_updates: Dict[str, Any],
    ) -> Dict[str, Any]:
        state = self.memory.get_accountability_state(content_plan_id)
        if not state:
            raise ValueError(f"No accountability state found for {content_plan_id}")

        plan = self.memory.get_content_plan(content_plan_id)
        if not plan:
            raise ValueError(f"No content plan found for {content_plan_id}")

        now = now_utc()
        tasks = plan["tasks"]

        # 1. Update tasks
        completed_ids = set(user_updates.get("completed_task_ids", []))
        for t in tasks:
            if t["task_id"] in completed_ids:
                t["status"] = "done"
                t["last_update"] = now.isoformat(timespec="minutes")

        blockers = user_updates.get("blockers", "")
        energy = user_updates.get("energy_level", None)

        # 2. Missed tasks -> failure patterns
        for t in tasks:
            try:
                # Due time uses 'seconds' from StrategistAgent
                due = datetime.fromisoformat(t["due_time"])
            except Exception:
                continue
            if t["status"] != "done" and now > due:
                trigger = f"{t['description']} after {due.hour}:00"
                self.memory.increment_failure_pattern(
                    trigger,
                    description=f"User tends to miss '{t['description']}' when scheduled after {due.hour}:00",
                )

        # 3. Completed?
        all_done = all(t["status"] == "done" for t in tasks)
        if all_done:
            plan["status"] = "completed"
            state["completed"] = True
        else:
            interval_h = state.get("check_in_interval_hours", 6)
            state["next_check_in"] = (now + timedelta(hours=interval_h)).isoformat(timespec="minutes")

        # Persist task updates
        plan["tasks"] = tasks
        # We'll append a compact checkin record to the plan's checkins list
        # 4. Coaching
        failure_patterns = self.memory.list_failure_patterns()
        prompt = f"""
You are the Accountability Agent for a solo creator.

CURRENT PLAN:
{plan}

FAILURE PATTERNS:
{failure_patterns}

USER UPDATE:
Blockers: {blockers}
Energy level (1-5): {energy}

1. Summarize their current progress.
2. Give 2-3 specific, kind but firm coaching suggestions.
3. Mention any repeating patterns you see.

Be concise but actionable.
"""
        coaching = llm_call(prompt)
        # record a short excerpt for history
        coaching_excerpt = coaching.strip().replace("\n", " ")[:500]

        self.memory.append_checkin(
            plan["id"],
            {
                "timestamp": now.isoformat(timespec="minutes"),
                "completed_ids": list(completed_ids),
                "blockers": blockers,
                "energy": energy,
                "coaching_excerpt": coaching_excerpt,
            },
        )

        # Save updated plan & state
        self.memory.update_content_plan(plan["id"], plan)
        self.memory.upsert_accountability_state(state)

        # log
        self.log("Check-in + coaching", coaching)

        return {
            "plan_status": plan["status"],
            "tasks": tasks,
            "coaching": coaching,
            "next_check_in": state.get("next_check_in"),
            "checkins": plan.get("checkins", []),
        }

### 🔁 Orchestrator and Full Planning Cycle

These functions manage the **Sequential Agent** flow:
1.  Ensures the User Profile is set (`ensure_user_profile`).
2.  Executes the full planning process (`run_full_planning_cycle`) by running the **TrendAgent** and **MemoryAgent** conceptually in parallel, feeding their results to the **StrategistAgent**, and initializing the **AccountabilityAgent** state via the A2A handoff.

In [6]:
# Cell 6: Orchestrator helpers

GLOBAL_MEMORY = MemoryBank()  # shared across app lifetime


def ensure_user_profile(
    name: str,
    niche: str,
    platform: str = "youtube",
    upload_goal_per_week: int = 1,
) -> Dict[str, Any]:
    existing = GLOBAL_MEMORY.get_user_profile()
    if existing:
        existing.update(
            {
                "name": name,
                "niche": niche,
                "platform": platform,
                "upload_goal_per_week": upload_goal_per_week,
            }
        )
        GLOBAL_MEMORY.set_user_profile(existing)
        return existing

    profile = {
        "name": name,
        "niche": niche,
        "platform": platform,
        "upload_goal_per_week": upload_goal_per_week,
        "preferred_length": "medium",
        "work_hours": {"start": "18:00", "end": "23:00"},
        "known_pain_points": [],
    }
    GLOBAL_MEMORY.set_user_profile(profile)
    return profile


def run_full_planning_cycle(
    name: str,
    niche: str,
    platform: str = "youtube",
    time_window_hours: int = 48,
) -> Dict[str, Any]:
    """
    1. Ensure user profile in memory.
    2. Run TrendAgent + MemoryAgent ("parallel").
    3. Run StrategistAgent; store content plan.
    4. Initialize AccountabilityAgent state.
    """
    user_profile = ensure_user_profile(name, niche, platform)

    trend_agent = TrendAgent("TrendAgent", GLOBAL_MEMORY)
    memory_agent = MemoryAgent("MemoryAgent", GLOBAL_MEMORY)
    strategist = StrategistAgent("StrategistAgent", GLOBAL_MEMORY)
    accountability = AccountabilityAgent("AccountabilityAgent", GLOBAL_MEMORY)

    trend_result = trend_agent.run(user_profile["niche"], user_profile["platform"])
    memory_result = memory_agent.run()

    handoff = strategist.run(trend_result, memory_result, time_window_hours)
    accountability.initialize_state(handoff)

    return handoff

### 🖥️ Gradio UI and Agent Deployment (Simulation)

This cell deploys the Gradio user interface, providing the interactive layer for the agent system. The UI demonstrates:
* Clear visualization of the `MemoryBank` data (tasks, history).
* The **Loop Agent** process via the "Check-in" tab.
* **Observability** via the "Logs" tab.

The launch of this UI via Colab/Kaggle Notebook simulates a basic **Agent Deployment**.

In [ ]:
# Cell 7: Gradio UI – Improved UX with check-in history (Final Fix for Markdown Error)

import gradio as gr
from datetime import datetime

# helper: format tasks for Dataframe (list-of-lists)
def tasks_to_table_rows(tasks):
    rows = []
    for t in tasks:
        # Added emoji for visual status
        status_emoji = "◻️" if t.get("status") == "pending" else "✅"
        # FIX: Using single quotes to avoid SyntaxError
        description_with_emoji = f"{status_emoji} {t.get('description')}"
        rows.append([t.get("task_id"), description_with_emoji, t.get("status"), t.get("due_time")])
    return rows

# helper: compute progress percent
def compute_progress(tasks):
    if not tasks:
        return 0
    done = sum(1 for t in tasks if t.get("status") == "done")
    total = len(tasks)
    return int((done / total) * 100)

# helper: render a simple ASCII-like progress bar string
def render_progress_bar(pct):
    filled = int((pct / 100) * 20)
    bar = "[" + "#" * filled + "-" * (20 - filled) + f"] {pct}%"
    return bar

# New helper: get ID of the most recently created plan
def get_latest_plan_id():
    plans = GLOBAL_MEMORY.list_content_plans()
    if plans:
        return plans[-1]["id"]
    return None

# UI helper functions (these reuse previously defined orchestrator/agents/memory)
def ui_generate_plan(name, niche, platform, hours):
    if not name or not niche:
        # FIX: Return raw string for the Markdown output (first item)
        return "Please enter your name and niche.", gr.update(value=[], headers=["task_id","description","status","due_time"]), gr.update(choices=[], value=[], interactive=False), gr.update(value=""), gr.update(choices=[], value=None), gr.update(value="")

    try:
        hours_int = int(hours)
    except Exception:
        hours_int = 48

    handoff = run_full_planning_cycle(
        name=name,
        niche=niche,
        platform=platform,
        time_window_hours=hours_int,
    )

    plan_id = handoff["content_plan_id"]
    plan = GLOBAL_MEMORY.get_content_plan(plan_id)

    if not plan:
        # FIX: Return raw string for the Markdown output (first item)
        return "Plan creation failed. Try again.", gr.update(value=[], headers=["task_id","description","status","due_time"]), gr.update(choices=[], value=[], interactive=False), gr.update(value=""), gr.update(choices=[], value=None), gr.update(value="")

    summary_md = f"""### ✅ New Content Plan Created\n
**Plan ID:** `{plan_id}`
**Topic:** **{plan['topic']}**
**Deadline (approx):** {plan['deadline']}

> Use the Plan ID above in the *Check-in* tab to report progress or pick from *All Plans*.
"""
    task_rows = tasks_to_table_rows(plan["tasks"])
    checklist_descs = [t["description"] for t in plan["tasks"]]
    plan_summary_label = f"{plan_id} — {plan['topic']} (status: {plan.get('status','unknown')})"

    all_plans_choices, _ = ui_list_plans()

    # 🌟 FIX: Return raw Markdown string (summary_md) for the first output
    return (summary_md,
            gr.update(value=task_rows),
            gr.update(choices=checklist_descs, value=[], interactive=True),
            gr.update(value=plan_summary_label),
            gr.update(choices=all_plans_choices, value=plan_summary_label),
            gr.update(value=plan_id))


def ui_list_plans():
    plans = GLOBAL_MEMORY.list_content_plans()
    if not plans:
        return [], "No plans found yet. Create a plan first."

    dropdown_items = []
    for p in reversed(plans):  # newest first
        label = f"{p['id']} — {p['topic']} (status: {p.get('status','unknown')})"
        dropdown_items.append(label)
    return dropdown_items, "Select a plan to view details and check-in."

def ui_view_plan(plan_label):
    if not plan_label:
        # FIX: Return raw string for the Markdown output (first item)
        return "Please select a plan.", [], gr.update(value="0%"), "", ""

    plan_id = plan_label.split(" — ")[0]
    plan = GLOBAL_MEMORY.get_content_plan(plan_id)
    if not plan:
        # FIX: Return raw string for the Markdown output (first item)
        return "Plan not found.", [], gr.update(value="0%"), "", ""

    progress_pct = compute_progress(plan["tasks"])
    progress_bar = render_progress_bar(progress_pct)

    md = f"""### 📋 Plan Details\n**Plan ID:** `{plan['id']}`\n**Topic:** **{plan['topic']}**\n**Created:** {plan['created_at']}\n**Deadline:** {plan['deadline']}\n**Status:** **{plan['status']}**\n\n**Progress:** {progress_bar}\n"""

    # Build check-in timeline markdown
    checkins = plan.get("checkins", [])
    if checkins:
        timeline_lines = ["\n### ⏱️ Check-in History"]
        for c in reversed(checkins):  # newest first
            ts = c.get("timestamp")
            completed = c.get("completed_ids", [])
            blockers = c.get("blockers", "")
            energy = c.get("energy", None)
            coach = c.get("coaching_excerpt", "")
            timeline_lines.append(f"  - **{ts}** — completed: `{completed}` — energy: {energy} — blockers: {blockers}")
            if coach:
                timeline_lines.append(f"    - coaching: {coach}")
        timeline_md = "\n".join(timeline_lines)
    else:
        timeline_md = "\n\n_No check-ins recorded yet for this plan._"

    # FIX: Return raw string for the Markdown output (first item)
    return md + timeline_md, tasks_to_table_rows(plan["tasks"]), gr.update(value=str(progress_pct) + "%"), plan.get('seo_raw','(none)')[:1000], plan.get('outline_raw','(none)')[:1200]

def ui_check_in_from_checklist(plan_id, selected_task_descs, blockers, energy_level):
    if not plan_id:
        # FIX: Return raw string for the Markdown output (first item)
        return "Please select or enter a Plan ID.", gr.update(value=[], headers=["task_id","description","status","due_time"]), gr.update(value="0%"), gr.update(choices=[], value=[], interactive=True), gr.update(choices=[], value=None), gr.update(value="")

    p = GLOBAL_MEMORY.get_content_plan(plan_id)
    if not p:
        # FIX: Return raw string for the Markdown output (first item)
        return f"Plan ID {plan_id} not found.", gr.update(value=[], headers=["task_id","description","status","due_time"]), gr.update(value="0%"), gr.update(choices=[], value=[], interactive=True), gr.update(choices=[], value=None), gr.update(value="")

    # Convert descriptions back to task_ids
    desc_to_id = {t["description"]: t["task_id"] for t in p["tasks"]}
    completed_ids = [desc_to_id[d] for d in (selected_task_descs or []) if d in desc_to_id]

    try:
        energy = int(energy_level) if energy_level is not None else None
    except Exception:
        energy = None

    user_updates = {
        "completed_task_ids": completed_ids,
        "blockers": blockers or "",
        "energy_level": energy,
    }

    acc_agent = AccountabilityAgent("AccountabilityAgent", GLOBAL_MEMORY)
    try:
        result = acc_agent.run_check_in(plan_id, user_updates)
    except ValueError as e:
        # FIX: Return raw string for the Markdown output (first item)
        return f"Error: {e}", gr.update(value=[], headers=["task_id","description","status","due_time"]), gr.update(value="0%"), gr.update(choices=[], value=[], interactive=True), gr.update(choices=[], value=None), gr.update(value="")

    plan = GLOBAL_MEMORY.get_content_plan(plan_id)
    if not plan:
        # FIX: Return raw string for the Markdown output (first item)
        return "Plan missing after check-in.", gr.update(value=[], headers=["task_id","description","status","due_time"]), gr.update(value="0%"), gr.update(choices=[], value=[], interactive=True), gr.update(choices=[], value=None), gr.update(value="")

    status_md = f"### 📊 Plan Status\n**Status:** {result['plan_status']}  \n**Next Check-in (suggested):** {result['next_check_in']}"
    coaching_md = f"### 🧠 Coaching\n{result['coaching']}"
    combined_md = status_md + "\n\n" + coaching_md

    task_rows = tasks_to_table_rows(result["tasks"])
    progress_pct = compute_progress(result["tasks"])

    # Re-populate checklist with current tasks and their completion status
    current_checklist_choices = [t["description"] for t in plan["tasks"]]
    selected_in_checklist = [t["description"] for t in plan["tasks"] if t["status"] == "done"]

    updated_dropdown_choices, _ = ui_list_plans()

    # 🌟 FIX: Return raw Markdown string (combined_md) for the first output
    return (combined_md,
            gr.update(value=task_rows),
            gr.update(value=str(progress_pct) + "%"),
            gr.update(value=selected_in_checklist, choices=current_checklist_choices, interactive=True),
            gr.update(choices=updated_dropdown_choices, value=plan_id),
            gr.update(value=plan_id))

def ui_show_logs():
    logs = get_logs()
    if not logs:
        return "No logs yet. Run a plan and a check-in first."
    lines = ["### 🔍 Recent agent logs\n"]
    for log in logs[-25:]:
        ts = log.get("timestamp")
        agent = log.get("agent")
        inp = log.get("input")
        lines.append(f"- `{ts}` **{agent}** – *{inp}*")
    return "\n".join(lines)

def ui_get_user_profile_data():
    profile = GLOBAL_MEMORY.get_user_profile() or {}
    return profile.get("name", ""), profile.get("niche", ""), profile.get("platform", "youtube"), profile.get("upload_goal_per_week", 1)

def ui_save_profile(name, niche, platform, upload_goal):
    try:
        upload_goal_int = int(upload_goal)
        if upload_goal_int <= 0: raise ValueError("Upload goal must be positive.")
    except ValueError:
        return "Upload goal must be a positive integer."

    ensure_user_profile(name, niche, platform, upload_goal_int)
    return "User profile saved successfully!"

# Build the Gradio Blocks UI
with gr.Blocks(title="Creator Strategy + Optimizer (Improved UX + History)") as demo:
    gr.Markdown(
        """
# 🎬 Creator Strategy + Optimizer Agent

Features:
- Create a 48h content plan driven by Trend + Memory + Strategist agents
- Browse all plans, view details, and see a timeline of check-ins
- Check off tasks via a visual checklist and submit progress
- All check-ins are stored in the plan history so you can learn patterns
"""
    )

    # Plan Content tab
    with gr.Tab("Plan Content") as tab_plan_content:
        gr.Markdown(
            """## 🚀 Generate New Content Plan\n_Enter your details and let the AI agents craft your next content plan._\n"""
        )
        with gr.Row():
            with gr.Column(scale=3):
                name_in = gr.Textbox(label="Your Name", value="Creator", interactive=True)
                niche_in = gr.Textbox(label="Your Niche", value="personal finance for students", interactive=True)
                platform_in = gr.Dropdown(label="Platform", choices=["youtube", "blog", "podcast"], value="youtube", interactive=True)
                hours_in = gr.Number(label="Time window (hours)", value=48, precision=0, interactive=True)
                btn_plan = gr.Button("Generate Content Plan")
                plan_feedback = gr.Markdown()
            with gr.Column(scale=2):
                gr.Markdown("#### Preview: Tasks for the new plan")
                tasks_df = gr.Dataframe(headers=["task_id","description","status","due_time"], interactive=False)
                checklist = gr.CheckboxGroup(label="Quick checklist (check and go to Check-in tab to submit)", choices=[], interactive=False)
                recent_plan_id_output = gr.Textbox(label="Latest Plan ID", interactive=False, visible=False) # Hidden, used for internal state

        def plan_click_handler(name, niche, platform, hours):
            summary_md_update, task_rows, checklist_update, _, all_plans_dropdown_update, latest_plan_id_val = ui_generate_plan(name, niche, platform, hours)

            return (summary_md_update, task_rows, checklist_update,
                    latest_plan_id_val, # for recent_plan_id_output
                    all_plans_dropdown_update, # for plan_dropdown
                    gr.update(value=latest_plan_id_val) # for plan_id_in_checkin
                   )

        btn_plan.click(
            fn=plan_click_handler,
            inputs=[name_in, niche_in, platform_in, hours_in],
            outputs=[
                plan_feedback, tasks_df, checklist,
                recent_plan_id_output,
                gr.Dropdown(elem_id="plan_dropdown_all_plans"), # Target in All Plans tab
                gr.Textbox(elem_id="plan_id_in_checkin") # Target in Check-in tab
            ]
        )

    # All Plans tab
    with gr.Tab("All Plans") as tab_all_plans:
        gr.Markdown(
            """## 🗓️ View All Content Plans\n_Select a plan from the dropdown to see its details and check-in history._\n"""
        )
        with gr.Row():
            plan_dropdown = gr.Dropdown(label="All Plans (newest first)", choices=[], value=None, interactive=True, elem_id="plan_dropdown_all_plans")
            btn_refresh = gr.Button("Refresh Plans")
        with gr.Row():
            plan_detail_md = gr.Markdown()
        with gr.Row():
            plan_tasks_df = gr.Dataframe(headers=["task_id","description","status","due_time"], interactive=False)
            plan_progress = gr.Textbox(label="Progress (percent)", interactive=False)

        # Define Markdown components for SEO and Outline within Accordions
        with gr.Accordion("SEO Details", open=False):
            seo_display = gr.Markdown()
        with gr.Accordion("Content Outline", open=False):
            outline_display = gr.Markdown()

        def refresh_plans_handler():
            labels, msg = ui_list_plans()
            return gr.update(choices=labels, value=None), gr.update(value=msg)

        btn_refresh.click(fn=refresh_plans_handler, inputs=None, outputs=[plan_dropdown, plan_detail_md])

        def on_plan_select_handler(selected_label):
            if not selected_label:
                return gr.update(value="Select a plan"), gr.update(value=[], headers=["task_id","description","status","due_time"]), gr.update(value="0%"), gr.update(value=""), gr.update(value=""), gr.update(value="")

            md_output, task_rows, progress_pct, seo_content, outline_content = ui_view_plan(selected_label)
            plan_id = selected_label.split(' — ')[0]

            # FIX: Returns update objects/strings based on component type
            return (gr.update(value=md_output),
                    gr.update(value=task_rows),
                    progress_pct,
                    gr.update(value=seo_content),
                    gr.update(value=outline_content),
                    gr.update(value=plan_id))

        plan_dropdown.change(
            fn=on_plan_select_handler,
            inputs=[plan_dropdown],
            outputs=[
                plan_detail_md, plan_tasks_df, plan_progress,
                seo_display, outline_display, # Directly update accordions' content
                gr.Textbox(elem_id="plan_id_in_checkin") # Also update check-in tab
            ]
        )

    # Check-in tab
    with gr.Tab("Check-in") as tab_checkin:
        gr.Markdown(
            """## ✅ Check-in Your Progress\n_Report on completed tasks, blockers, and energy levels for your active plan._\n"""
        )
        with gr.Row():
            plan_id_in = gr.Textbox(label="Plan ID (from 'Plan Content' or 'All Plans')", elem_id="plan_id_in_checkin", interactive=True)
        with gr.Row():
            checklist_in = gr.CheckboxGroup(label="Tasks to mark as completed (select & submit)", choices=[], interactive=True)
        with gr.Row():
            blockers_in = gr.Textbox(label="Blockers (what got in the way?)", lines=2, interactive=True)
            energy_in = gr.Slider(minimum=1, maximum=5, step=1, value=3, label="Energy level (1–5)", interactive=True)
            btn_submit_checkin = gr.Button("Submit Check-in")
        with gr.Row():
            checkin_md = gr.Markdown()
        with gr.Row():
            checkin_tasks_df = gr.Dataframe(headers=["task_id","description","status","due_time"], interactive=False)
            checkin_progress = gr.Textbox(label="Progress (after check-in)", interactive=False)

        # When user types a plan id, populate checklist choices with descriptions
        def prepare_checklist_for_plan(plan_id):
            if not plan_id:
                return gr.update(choices=[], value=[], interactive=True)
            p = GLOBAL_MEMORY.get_content_plan(plan_id)
            if not p:
                return gr.update(choices=[], value=[], interactive=True)

            current_choices = [t["description"] for t in p["tasks"]]
            currently_completed = [t["description"] for t in p["tasks"] if t["status"] == "done"]
            return gr.update(choices=current_choices, value=currently_completed, interactive=True)

        plan_id_in.change(fn=prepare_checklist_for_plan, inputs=[plan_id_in], outputs=[checklist_in])

        def submit_checkin_handler(plan_id, chosen_descriptions, blockers, energy):
            if not plan_id:
                return gr.update(value="Please enter a Plan ID."), gr.update(value=[], headers=["task_id","description","status","due_time"]), gr.update(value="0%"), gr.update(choices=[], value=[], interactive=True), gr.update(choices=[], value=None), gr.update(value="")

            md_output, task_rows, progress_pct, updated_checklist_selected, all_plans_dropdown_update_value, checklist_choices_update = ui_check_in_from_checklist(plan_id, chosen_descriptions, blockers, energy)

            # FIX: Returns update objects/strings based on component type
            return (md_output, # Returned as raw string
                    gr.update(value=task_rows),
                    gr.update(value=progress_pct),
                    gr.update(value=updated_checklist_selected, choices=checklist_choices_update, interactive=True),
                    gr.Dropdown(elem_id="plan_dropdown_all_plans"), # Target All Plans dropdown
                    gr.Textbox(elem_id="plan_id_in_checkin")) # Maintain current plan_id

        btn_submit_checkin.click(
            fn=submit_checkin_handler,
            inputs=[plan_id_in, checklist_in, blockers_in, energy_in],
            outputs=[
                checkin_md, checkin_tasks_df, checkin_progress, checklist_in, # Current tab outputs
                gr.Dropdown(elem_id="plan_dropdown_all_plans"), # Target All Plans dropdown
                gr.Textbox(elem_id="plan_id_in_checkin") # Maintain current plan_id
            ]
        )

        # When tab is selected, populate plan_id_in with the latest plan if available
        @tab_checkin.select
        def load_latest_plan_into_checkin():
            latest_id = get_latest_plan_id()
            if latest_id:
                return gr.update(value=latest_id)
            return gr.update(value="")


    # User Profile Tab
    with gr.Tab("User Profile") as tab_user_profile:
        gr.Markdown(
            """## 👤 Manage Your Profile\n_Update your personal details to tailor content generation._\n"""
        )
        with gr.Column():
            profile_name_in = gr.Textbox(label="Name", interactive=True)
            profile_niche_in = gr.Textbox(label="Niche", interactive=True)
            profile_platform_in = gr.Dropdown(label="Platform", choices=["youtube", "blog", "podcast"], interactive=True)
            profile_upload_goal_in = gr.Number(label="Upload Goal per Week", precision=0, interactive=True)
            btn_save_profile = gr.Button("Save Profile")
            profile_save_status = gr.Markdown()

        @tab_user_profile.select
        def load_user_profile_data():
            name, niche, platform, upload_goal = ui_get_user_profile_data()
            return gr.update(value=name), gr.update(value=niche), gr.update(value=platform), gr.update(value=upload_goal)

        btn_save_profile.click(
            fn=ui_save_profile,
            inputs=[profile_name_in, profile_niche_in, profile_platform_in, profile_upload_goal_in],
            outputs=[profile_save_status]
        )

    # Logs tab
    with gr.Tab("Logs") as tab_logs:
        gr.Markdown(
            """## 📝 Agent Activity Log\n_See a trace of all agent interactions and outputs._\n"""
        )
        logs_btn = gr.Button("Refresh Logs")
        logs_md = gr.Markdown()
        logs_btn.click(fn=ui_show_logs, inputs=None, outputs=logs_md)

        @tab_logs.select
        def auto_refresh_logs():
            return ui_show_logs()

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ef761fd25feb605329.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/gradio/blocks.py:1891: UserWarning: A function (load_user_profile_data) returned too many output values (needed: 0, returned: 4). Ignoring extra values.
    Output components:
        []
    Output values returned:
        [{'value': 'TrueCrimeTime', '__type__': 'update'}, {'value': 'True Crime Reports', '__type__': 'update'}, {'value': 'youtube', '__type__': 'update'}, {'value': 1, '__type__': 'update'}]
  warnings.warn(
/tmp/ipython-input-3971887581.py:53: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow()
/tmp/ipython-input-3971887581.py:53: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow()
